In [1]:
import os
import sys
import shutil
import platform
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import flopy
from flopy.discretization import VertexGrid
from flopy.utils import Raster
from flopy.utils import GridIntersect
from flopy.utils.gridgen import Gridgen
gridgen_exe = "gridgen"
if platform.system() in "Windows":
    gridgen_exe += ".exe"
gridgen_exe = flopy.which("gridgen")
if gridgen_exe is None:
    msg = (
        "Warning, gridgen is not in your path. "
        "When you create the griden object you will need to "
        "provide a full path to the gridgen binary executable."
    )
    print(msg)
else:
    print("gridgen executable was found at: {}".format(gridgen_exe))

print(f"Pandas version = {pd.__version__}")
print(f"Numpy version = {np.__version__}")
print(f"Flopy version = {flopy.__version__}")
print(f"Matplotlib version = {matplotlib.__version__}")

gridgen executable was found at: C:\mffiles\gridgen.EXE
Pandas version = 2.1.1
Numpy version = 1.26.0
Flopy version = 3.4.3
Matplotlib version = 3.8.0


In [8]:
ws = os.path.join('Leapfrog_load') # here we are making a path not creating the folder
gis_f = os.path.join(ws,'GIS') # creating a sub-directory path for our gis input/output
model_f = os.path.join(ws,'model') # creating a sub-directory path for our model input/output
plots_f = os.path.join(ws,'plots') # creating a sub-directory path for our plots
for path in [ws,gis_f,model_f,plots_f]:
    if os.path.exists(path): # here we are asking if the path exists on the computer. 
        shutil.rmtree(path)# if it does exist, delete it and all the files in it
        os.mkdir(path) # then remake it
    else:
        os.mkdir(path) # if it doesn't exist then make the folder

In [9]:
path_to_files = os.path.join('files','Leapfrog_Export')
flist = [x for x in os.listdir(path_to_files)]
_ = [shutil.copyfile(os.path.join(path_to_files,file),os.path.join(model_f,file)) for file in flist]    

In [10]:
flist

['Leapfrog_Export.bas6',
 'Leapfrog_Export.dis',
 'Leapfrog_Export.list',
 'Leapfrog_Export.lpf',
 'Leapfrog_Export.nam',
 'Leapfrog_Export.zon',
 'Leapfrog_Export_conc.txt',
 'Leapfrog_Export_conc.ucn']

In [11]:
# Now look at the flow.dis input file
nam_file = 'Leapfrog_Export.nam'
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,nam_file))]

LIST 10 WB19BS1E0003_PT.list
DIS 11 WB19BS1E0003_PT.dis
BAS6 12 WB19BS1E0003_PT.bas6
LPF 13 WB19BS1E0003_PT.lpf
ZONE 14 WB19BS1E0003_PT.zon


The filenames in the .nam file do not match the rest of the names of the files in the directory so this load will fail. You will have to rename the files in the namfile to match those in the rest of the directory in order to get it to load. Unsure why Leapfrog did/does this. Or you could rename the files in the directory to match those listed in the namfile.

In [12]:
new_nam = 'WB19BS1E0003_PT'
_ = [os.rename(os.path.join(model_f,file),os.path.join(model_f,new_nam+"."+file.split(".")[-1])) for file in flist]    

In [13]:
# Load an MF2005 model
ml = flopy.modflow.Modflow.load(
    f"{new_nam}.nam",
    model_ws=model_f,
    verbose=True,
    version="mf2005",
    check=False,
)


Creating new model with name: WB19BS1E0003_PT
--------------------------------------------------

Parsing the namefile --> c:\Working\NCGRT_IntroFlopy\Leapfrog_load\model\WB19BS1E0003_PT.nam

--------------------------------------------------
External unit dictionary:
{10: filename:c:\Working\NCGRT_IntroFlopy\Leapfrog_load\model\WB19BS1E0003_PT.list, filetype:LIST, 11: filename:c:\Working\NCGRT_IntroFlopy\Leapfrog_load\model\WB19BS1E0003_PT.dis, filetype:DIS, 12: filename:c:\Working\NCGRT_IntroFlopy\Leapfrog_load\model\WB19BS1E0003_PT.bas6, filetype:BAS6, 13: filename:c:\Working\NCGRT_IntroFlopy\Leapfrog_load\model\WB19BS1E0003_PT.lpf, filetype:LPF, 14: filename:c:\Working\NCGRT_IntroFlopy\Leapfrog_load\model\WB19BS1E0003_PT.zon, filetype:ZONE}
--------------------------------------------------

ModflowBas6 free format:True

loading dis package file...
   Loading dis package with:
      17 layers, 50 rows, 75 columns, and 1 stress periods
   loading laycbd...
   loading delr...
   loa

In [20]:
dis = ml.get_package("dis")
dis.nlay

17

# Do a vtk export for viewing with paraview

In [21]:
ml.export(plots_f, fmt="vtk")